In [1]:
from langgraph.graph import StateGraph, MessagesState, START, END

## langgraph

In [2]:
def mock_llm(state: MessagesState):
    return {"messages": [{"role": "ai", "content": "hello world"}]}

graph = StateGraph(MessagesState)
graph.add_node(mock_llm)
graph.add_edge(START, "mock_llm")
graph.add_edge("mock_llm", END)
graph = graph.compile()

graph.invoke({"messages": [{"role": "user", "content": "Hi~"}]})

{'messages': [HumanMessage(content='Hi~', additional_kwargs={}, response_metadata={}, id='b2edd17a-afcf-425c-82fc-2f92ec2a8c36'),
  AIMessage(content='hello world', additional_kwargs={}, response_metadata={}, id='506b97f2-bb9d-49c9-83cb-5c6ccfe42b17', tool_calls=[], invalid_tool_calls=[])]}

## 测试qwen (langchain)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
 
llm = ChatOpenAI(
    # 根据自己的需求配置，可以是环境变量，也可以是文本内容
    api_key="sk-***", 
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    model="qwen-plus"
)
 
messages = [
    # 系统提示词：将英文翻译为中文
    SystemMessage("Translate the following from English into chinese"),
    HumanMessage("hi! I'm a PhD student"),
]
 
aimessages = llm.invoke(messages)
print(aimessages.content)

嗨！我是一名博士生。


## langgraph + qwen

In [4]:
def qwen_llm(state: MessagesState):
    messages = state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}

graph = StateGraph(MessagesState)
graph.add_node("qwen", qwen_llm)
graph.add_edge(START, "qwen")
graph.add_edge("qwen", END)
graph = graph.compile()

graph.invoke({"messages": [{"role": "user", "content": "Hi~ Who are you?"}]})

{'messages': [HumanMessage(content='Hi~ Who are you?', additional_kwargs={}, response_metadata={}, id='df9fa8f3-a413-4f84-a616-eaa336db1048'),
  AIMessage(content="Hello! I'm Qwen, a large-scale language model independently developed by the Tongyi Lab under Alibaba Group. I can assist you with answering questions, writing, logical reasoning, programming, and more. I can also engage in casual conversations or help with work and study tasks. Feel free to let me know if you need any assistance! 😊", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 14, 'total_tokens': 84, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'qwen-plus', 'system_fingerprint': None, 'id': 'chatcmpl-cf73a7da-9ece-9e4a-9d46-68d8187aeab2', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bbc52-de5b-73a0-b158-5b8b4fd81858-0', tool_calls=[], invali

## complex langgraph structure

### 多节点，简单边（双 agent）

In [5]:
# 算数 agent
def math_agent(state: MessagesState):
    system_prompt = SystemMessage("""
    你是一个普通的计算员，会做基本的数学运算。
    请根据用户的问题或者审核员的建议，直接给出答案。不要给出计算过程。
    """)
    messages = [system_prompt] + state["messages"]
    response = llm.invoke(messages)

    # 标记 response 名称
    response.name = "math_agent"
    return {"messages": [response]}


# 检查 agent
def check_agent(state: MessagesState):
    system_prompt = SystemMessage("""
    你是一个数学家，擅长检查各种数学问题。
    请检查 math_agent 的答案是否正确。
    如果正确，请给出 "correct"，并确保回答中没有"incorrect"这个词。
    如果错误，请给出新的计算建议，请给出 "incorrect"。
    """)
    messages = [system_prompt] + state["messages"]
    response = llm.invoke(messages)

    # 标记 response 名称
    response.name = "check_agent"
    return {"messages": [response]}



graph = StateGraph(MessagesState)
graph.add_node("math_agent", math_agent)
graph.add_node("check_agent", check_agent)
graph.add_edge(START, "math_agent")
graph.add_edge("math_agent", "check_agent")
graph.add_edge("check_agent", END)
graph = graph.compile()

# 生成图的流程
qa_async_ascii = graph.get_graph().print_ascii()
print(qa_async_ascii)


graph.invoke({"messages": [{"role": "user", "content": "1 + 1 = ?"}]})

 +-----------+   
 | __start__ |   
 +-----------+   
        *        
        *        
        *        
+------------+   
| math_agent |   
+------------+   
        *        
        *        
        *        
+-------------+  
| check_agent |  
+-------------+  
        *        
        *        
        *        
  +---------+    
  | __end__ |    
  +---------+    
None


{'messages': [HumanMessage(content='1 + 1 = ?', additional_kwargs={}, response_metadata={}, id='71bede86-edd0-4259-a50f-b924656b9fb4'),
  AIMessage(content='2', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 56, 'total_tokens': 57, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'qwen-plus', 'system_fingerprint': None, 'id': 'chatcmpl-703f61e2-82f2-958f-8f18-12858a6361c6', 'finish_reason': 'stop', 'logprobs': None}, name='math_agent', id='lc_run--019bbc52-e4fc-79c0-89e1-bef0b3cc7120-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 56, 'output_tokens': 1, 'total_tokens': 57, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}),
  AIMessage(content='correct', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 78, 'total_to

In [6]:
graph.invoke({"messages": [{"role": "user", "content": "what is 3^{12345} (mod 100)?"}]})

{'messages': [HumanMessage(content='what is 3^{12345} (mod 100)?', additional_kwargs={}, response_metadata={}, id='dc6a0cdd-3eee-41ba-81ff-354a6f25254d'),
  AIMessage(content='81', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 68, 'total_tokens': 70, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'qwen-plus', 'system_fingerprint': None, 'id': 'chatcmpl-aa6922a0-026b-9518-b185-02ee907ee09f', 'finish_reason': 'stop', 'logprobs': None}, name='math_agent', id='lc_run--019bbc52-e968-7831-8c53-5885c69e5547-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 68, 'output_tokens': 2, 'total_tokens': 70, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}),
  AIMessage(content="incorrect\n\nThe value of $ 3^{12345} \\mod 100 $ cannot be correctly computed by simple observation due to the larg

### 多节点，判断边（双 agent）

In [7]:
def should_continue(state: MessagesState):
    """
    根据审核结果，判断是否重新计算。
    返回下一个节点的名称。
    """
    last_message = state["messages"][-1]
    if 'incorrect' in last_message.content:
        print("审核不通过，打回中...")
        return "math_agent"
    else:
        print("审核通过，返回结果...")
        return "END"


In [8]:
workflow = StateGraph(MessagesState)

# 添加节点
workflow.add_node("math_agent", math_agent)
workflow.add_node("check_agent", check_agent)

# 添加边
workflow.add_edge(START, "math_agent")
workflow.add_edge("math_agent", "check_agent")
# 复杂判断
workflow.add_conditional_edges(
    "check_agent",
    should_continue,
    {
        "math_agent": "math_agent",
        "END": END,
    },
)
workflow = workflow.compile()

workflow.invoke({"messages": [{"role": "user", "content": "what is 3^{12345} (mod 100)?"}]})

审核不通过，打回中...
审核通过，返回结果...


{'messages': [HumanMessage(content='what is 3^{12345} (mod 100)?', additional_kwargs={}, response_metadata={}, id='d7c5f388-ef79-4579-8a95-0c9a00966ef8'),
  AIMessage(content='81', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 68, 'total_tokens': 70, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'qwen-plus', 'system_fingerprint': None, 'id': 'chatcmpl-ecf2bd03-29c8-94a8-ad04-639f7bad62dd', 'finish_reason': 'stop', 'logprobs': None}, name='math_agent', id='lc_run--019bbc53-1eb3-7e82-a50a-1c62ffd288c0-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 68, 'output_tokens': 2, 'total_tokens': 70, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}),
  AIMessage(content="incorrect\n\nThe claim that $ 3^{12345} \\mod 100 = 81 $ is incorrect.\n\nLet’s compute $ 3^{12345} \\mod 100 $ prop

In [9]:
# 生成图的流程图片
qa_async_ascii = workflow.get_graph().print_ascii()
print(qa_async_ascii)

 +-----------+   
 | __start__ |   
 +-----------+   
        *        
        *        
        *        
+------------+   
| math_agent |   
+------------+   
        .        
        .        
        .        
+-------------+  
| check_agent |  
+-------------+  
        .        
        .        
        .        
  +---------+    
  | __end__ |    
  +---------+    
None
